<a href="https://colab.research.google.com/github/tjdux/basic_of_ml/blob/main/28_%EA%B0%95%EC%95%84%EC%A7%80%EC%99%80_%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%82%AC%EC%A7%84_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8%EC%9D%98_%EC%84%B1%EB%8A%A5_%EA%B0%9C%EC%84%A0%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 훈련 성능을 높이는 CNN 모델 - ResNet
- 역전파
  - 각 유닛이 예측과 정답 사이의 오차에 얼마나 기여하는지 모델의 끝에서부터 앞으로 누적하여 계산
  - 1️⃣ 오차의 발생 원인을 찾고, 2️⃣ 신경망의 가중치를 조정하여 예측을 점점 더 정확하게 만드는 역할
- 딥러닝 모델의 문제점: 신경망이 깊어질수록 가중치를 변경해 오차를 줄이는 **그레이디언트**(gradient)가 점점 작아지기 때문에 입력 부분에 가까운 층의 가중치가 잘 변경되지 않음
- **잔차 블록**(residual block)
  - 입력을 출력에 직접 연결하는 **스킵 연결**을 추가해 그레이디언트 소실 문제를 완화
  

## ResNet50 모델 만들기
- **병목 블록**(bottleneck block)
  - 첫 번째 합성곱층: 입력의 공간 방향 크기를 줄임
  - 두 번째 합성곱층: 입력 채널의 크기를 유지한 채 특징을 추출
  - 세 번째 합성곱층: 채널 수를 확장
- **잔차 모듈**(residual module)
  - 스킵 연결을 통해 입력 데이터를 직접 다음 층으로 전달
  - 신경망이 깊어지더라도 학습이 가능하도록 돕는 잔차 블록과 그 잔차 블록이 모인 잔차 스택으로 구성

### 배치 정규화
- 잔차 블록 내에서 학습의 속도를 높이고 모델의 안정성을 개선

In [1]:
# 입력 준비 및 배치 정규화 적용하기
import keras
from keras import layers

inputs = layers.Input(shape=(224, 224, 3))
x = layers.ZeroPadding2D(padding=3)(inputs) # 패딩 추가 -> (230, 230, 3)
x = layers.Conv2D(filters=64, kernel_size=7, strides=2)(x) #(112, 112, 64)
x = layers.BatchNormalization(epsilon=1e-5)(x)
x = layers.Activation('relu')(x)

- **배치 정규화**(batch normalization)
  - 각 층의 출력을 배치 단위로 다시 정규화함으로써 훈련의 속도와 성능을 높일 수 있는 방법
  - 각 입력의 값에 적용하는 연산이므로 입력과 출력의 차원이 동일
  - 훈련하는 동안 각 배치의 평균과 분산의 이동 평균을 구하여 훈련이 끝난 후 예측을 수행할 때 사용

In [2]:
# 패딩 추가 및 최대 풀링 적용
x = layers.ZeroPadding2D(padding=1)(x) #(114, 114, 64)
x = layers.MaxPooling2D(3, strides=2)(x) #(56, 56, 64)

### 잔차 스택 만들기
- 4개의 잔차 스택으로 구성
- 각 스택에 3개, 4개, 6개, 3개의 잔차 블록이 들어감

In [3]:
def build_stack(x):
  # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1 사용
  x = residual_stack(x, 3, 64, first_stride=1)
  # 두 번째 ~ 네 번째 잔차 블록
  for blocks, filters in [(4, 128), (6, 256), (3, 512)]:
    x = residual_stack(x, blocks, filters, first_stride=2)
  return x

def residual_stack(x, blocks, filters, first_stride=2):
  # 첫 번째 잔차 블록은 합성곱 스킵 연결을 사용하고
  # 첫 번째 잔차 블록의 첫 번째 합성곱 스트라이드는 first_stride
  x = residual_block(x, filters, first_stride=first_stride, conv_skip=True)
  for _ in range(1, blocks):
    # 나머지 잔차 블록의 첫 번째 합성곱 스트라이드는 1
    x = residual_block(x, filters, first_stride=1, conv_skip=False)
  return x

- 1X1 합성곱 또는 **점별 합성곱**(pointwise convolution)은 공간 방향 차원을 유지하면서 채널 차원을 변경하기 위해 자주 사용

In [5]:
def residual_block(x, filters, first_stride=1, conv_skip=False):
  skip_conn = x

  # 합성곱과 배치 정규화, 렐루 활성화
  # 1x1, filters개 필터, 스트라이드는 first_stride에 따라 1 또는 2
  x = layers.Conv2D(filters=filters, kernel_size=1,
                    strides=first_stride)(x)
  x = layers.BatchNormalization(epsilon=1e-5)(x)
  x = layers.Activation('relu')(x)

  # 3x3, filters개 필터
  x = layers.Conv2D(filters=filters, kernel_size=3,
                    padding='same')(x)
  x = layers.BatchNormalization(epsilon=1e-5)(x)
  x = layers.Activation('relu')(x)

  # 1x1, filters*4개 필터
  x = layers.Conv2D(filters=filters*4, kernel_size=1)(x)
  x = layers.BatchNormalization(epsilon=1e-5)(x)

  # conv_skip=True라면 1x1 합성곱을 사용해 채널 크기 filters*4개로 늘리기
  if conv_skip:
    skip_conn = layers.Conv2D(filters=filters*4, kernel_size=1,
                              strides=first_stride)(skip_conn)
    skip_conn = layers.BatchNormalization(epsilon=1e-5)(skip_conn)
  x = layers.Add()([skip_conn, x])
  x = layers.Activation('relu')(x)

  return x

### ResNet 모델 만들기

In [6]:
x = build_stack(x)

In [7]:
x = layers.GlobalAveragePooling2D()(x) # 전역 풀링층: 각각의 특성 맵을 하나의 값으로 요약
outputs = layers.Dense(100, activation='softmax')(x)

In [8]:
model = keras.Model(inputs, outputs)

In [9]:
# ResNet101 모델 만들기
def build_stack101(x):
  # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1 사용
  x = residual_stack(x, 3, 64, first_stride=1)
  # 두 번째 ~ 네 번째 잔차 블록
  for blocks, filters in [(4, 128), (23, 256), (3, 512)]:
    x = residual_stack(x, blocks, filters, first_stride=2)
  return x

In [10]:
# ResNet152 모델 만들기
def build_stack152(x):
  # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1 사용
  x = residual_stack(x, 3, 64, first_stride=1)
  # 두 번째 ~ 네 번째 잔차 블록
  for blocks, filters in [(8, 128), (36, 256), (3, 512)]:
    x = residual_stack(x, blocks, filters, first_stride=2)
  return x

## 강아지와 고양이 사진 분류하기

In [11]:
!gdown 1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
!unzip cat-dog-images.zip

Downloading...
From: https://drive.google.com/uc?id=1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
To: /content/cat-dog-images.zip
100% 182k/182k [00:00<00:00, 138MB/s]
Archive:  cat-dog-images.zip
   creating: images/
  inflating: images/dog.png          
  inflating: images/cat.png          


In [12]:
from PIL import Image
import numpy as np
from keras.applications import resnet

dog_png = Image.open('images/dog.png')
resnet_prep_dog = resnet.preprocess_input(np.array(dog_png))

In [13]:
resnet50 = keras.applications.ResNet50()
predictions = resnet50.predict(resnet_prep_dog[np.newaxis, :])

resnet.decode_predictions(predictions)

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


[[('n02099712', 'Labrador_retriever', np.float32(0.3853529)),
  ('n02099601', 'golden_retriever', np.float32(0.089699574)),
  ('n02100735', 'English_setter', np.float32(0.04212416)),
  ('n02106166', 'Border_collie', np.float32(0.0377744)),
  ('n02101388', 'Brittany_spaniel', np.float32(0.030700417))]]

In [14]:
cat_png = Image.open('images/cat.png')
resnet_prep_cat = resnet.preprocess_input(np.array(cat_png))

predictions = resnet50.predict(resnet_prep_cat[np.newaxis, :])

resnet.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


[[('n02123045', 'tabby', np.float32(0.86861)),
  ('n02124075', 'Egyptian_cat', np.float32(0.05077501)),
  ('n02123159', 'tiger_cat', np.float32(0.04256705)),
  ('n07930864', 'cup', np.float32(0.0027631535)),
  ('n03443371', 'goblet', np.float32(0.0020991666))]]

## GoogleLeNet
- **인셉션 모듈**(inception module) 사용 👉 다양한 크기의 합성곱(1x1, 3x3, 5x5)을 병렬로 사용해 여러 스케일에서 특징을 추출하여 보다 효율적으로 이미지를 분석

### GoogleLeNet으로 강아지와 고양이 사진 분류하기

In [15]:
from keras.utils import load_img
from keras.applications import inception_v3

dog_png = load_img('images/dog.png', target_size=(299, 299)) # 사이즈 변경
incep_prep_dog = inception_v3.preprocess_input(np.array(dog_png))

In [16]:
inception = keras.applications.InceptionV3()

predictions = inception.predict(incep_prep_dog[np.newaxis, :])
inception_v3.decode_predictions(predictions)

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


[[('n02104029', 'kuvasz', np.float32(0.13835052)),
  ('n02099712', 'Labrador_retriever', np.float32(0.077772975)),
  ('n02106166', 'Border_collie', np.float32(0.07198338)),
  ('n02111500', 'Great_Pyrenees', np.float32(0.066149026)),
  ('n02099601', 'golden_retriever', np.float32(0.02838335))]]

In [17]:
cat_png = load_img('images/cat.png', target_size=(299, 299))
incep_prep_cat = inception_v3.preprocess_input(np.array(cat_png))
predictions = inception.predict(incep_prep_cat[np.newaxis, :])
inception_v3.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


[[('n02124075', 'Egyptian_cat', np.float32(0.68673694)),
  ('n02123159', 'tiger_cat', np.float32(0.13262996)),
  ('n02123045', 'tabby', np.float32(0.04215028)),
  ('n04040759', 'radiator', np.float32(0.0016103373)),
  ('n02971356', 'carton', np.float32(0.0011297752))]]

In [21]:
from keras.applications import inception_resnet_v2 as incep_res_v2

incep_res_prep_dog = incep_res_v2.preprocess_input(np.array(dog_png))
inception_resnet = keras.applications.InceptionResNetV2()
predictions = inception_resnet.predict(incep_res_prep_dog[np.newaxis, :])
incep_res_v2.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


[[('n02099712', 'Labrador_retriever', np.float32(0.6563322)),
  ('n02104029', 'kuvasz', np.float32(0.13956271)),
  ('n02099601', 'golden_retriever', np.float32(0.055945188)),
  ('n02111500', 'Great_Pyrenees', np.float32(0.048894834)),
  ('n02100735', 'English_setter', np.float32(0.002117878))]]

In [22]:
incep_res_prep_cat = incep_res_v2.preprocess_input(np.array(cat_png))

predictions = inception_resnet.predict(incep_res_prep_cat[np.newaxis, :])
incep_res_v2.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


[[('n02123045', 'tabby', np.float32(0.42494813)),
  ('n02124075', 'Egyptian_cat', np.float32(0.25831005)),
  ('n02123159', 'tiger_cat', np.float32(0.12795246)),
  ('n02127052', 'lynx', np.float32(0.0034486083)),
  ('n04525038', 'velvet', np.float32(0.0024461073))]]